# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import os
import pandas as pd
import json

In [2]:
response = requests.get('http://api.citybik.es/v2/networks')
print(response.status_code)

200


In [3]:
def get_city_bike_data(city_name):
    url = 'http://api.citybik.es/v2/networks/'
   
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        networks = data.get('networks',[])
        for network in networks:
            if network['location']['city'].lower() == city_name.lower():
                return network
        print(f"'{city_name}' not found in the CityBikes API.")
        return None
    else:
        print(f"Error: Unable to fetch data for {city_name}. Status code: {response.status_code}")
        return None

In [4]:
city_name = 'Vancouver'
city_bike_data = get_city_bike_data(city_name)

if city_bike_data is not None:
    print(json.dumps(city_bike_data, indent=4))

{
    "company": [
        "Vanncouver Bike Share Inc.",
        "CycleHop LLC",
        "City of Vancouver",
        "Shaw Communications Inc.",
        "Fifteen"
    ],
    "gbfs_href": "https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json",
    "href": "/v2/networks/mobibikes",
    "id": "mobibikes",
    "location": {
        "city": "Vancouver",
        "country": "CA",
        "latitude": 49.2827,
        "longitude": -123.1207
    },
    "name": "Mobi"
}


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [8]:
def get_bike_station_details(city_name):
    url = f'http://api.citybik.es/v2/networks/{network_id}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get('network', None)
    else:
        print(f"Error: Unable to fetch data for {network_id}. Status code: {response.status_code}")
        return None
    
network_id = 'mobibikes'
bike_station_details = get_bike_station_details(network_id)
    
if bike_station_details is not None:
    stations = bike_station_details.get('stations', [])
    if not stations:
        print(f"No bike stations data available for {network_id}.")
    else:
        station_details_list = []
        for station in stations:
            station_name = station['name']
            latitude = station['latitude']
            longitude = station['longitude']
            num_bikes = station['free_bikes']
            empty_slots = station['empty_slots']
            station_id = station['id']
            
            station_details_list.append({
                'Station Name': station_name,
                # 'Station ID': station_id,   #didn't want to show it to cluster data...
                'Latitude': latitude,
                'Longitude': longitude,
                'Available Bikes': num_bikes,
                'Empty Slots': empty_slots
            })
        df = pd.DataFrame(station_details_list)
else:
    print(f"No data available for {network_id} or network ID not found.")

Put your parsed results into a DataFrame.

In [9]:
df

,Station Name,Station ID,Latitude,Longitude,Available Bikes,Empty Slots
0,10th & Cambie,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,16,19
1,Yaletown-Roundhouse Station,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,9,7
2,Dunsmuir & Beatty,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,10,15
3,12th & Yukon (City Hall),66f873d641d448bd1572ab086665a458,49.260599,-123.113504,9,6
4,8th & Ash,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,2,14
...,...,...,...,...,...,...
240,Cordova & Granville,aa1cbf97abc3cd50515bc54633a9bb2a,49.285670,-123.112543,16,2
241,22nd & Main,13021bba7bbeca1ccd8726ca17da59e6,49.250940,-123.101306,5,9
242,PNE - Hastings & Windermere,a74744ce4bb7ea2aa9f406ac8bff95d8,49.280977,-123.035969,6,34
243,1st & Fir,5699b40126e10c2f68eefc3eb18ff3a1,49.270783,-123.141564,0,20
